# Convolutional Neural Network

## Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.preprocessing import image

# Tensorflow version
print(tf.__version__)


# Part 1 - Data Preprocessing


## Training Data Set

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
    "./flower_photos/train",
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical')

## Validation Data Set

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

valid_set = valid_datagen.flow_from_directory('./flower_photos/validation',
                                                target_size=(128, 128),
                                                batch_size=32,
                                                class_mode='categorical')


# Part 2 - Building the CNN

### Initializing the model

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, 
                                kernel_size=3, 
                                activation='relu', 
                                input_shape=[128, 128, 3]))

### Step 2 - Pooling Layer

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding 2nd Convolution Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding 3rd Convolutional Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding 4th Convolution Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding 5th Convolution Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Flattening


In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
# Full connected layer
cnn.add(tf.keras.layers.Dense(units=200, activation='relu'))

# 2nd Full connected layer
cnn.add(tf.keras.layers.Dense(units=150, activation='relu'))

# Output layer
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))

### Compiling the CNN

In [ ]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Plotting accuracy and loss graphs

In [ ]:
# Custom callback to record accuracy and loss values after each epoch
class LossAccHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.accuracy = []
        self.val_losses = []
        self.val_accuracy = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_accuracy.append(logs.get('val_accuracy'))

In [ ]:
# Training the CNN with custom callback
history = LossAccHistory()
cnn.fit(x=training_set,validation_data=valid_set,epochs=40,callbacks=[history])

In [ ]:
epochs = range(1, len(history.accuracy) + 1)

plt.figure(figsize=(10, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, history.accuracy, label='Training Accuracy', marker='o')
plt.plot(epochs, history.val_accuracy, label='Validation Accuracy', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(epochs, history.losses, label='Training Loss', marker='o')
plt.plot(epochs, history.val_losses, label='Validation Loss', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Saving the model


In [ ]:
cnn.save('Flowers_final.h5')

# Testing

### Loading the model

In [ ]:
model = tf.keras.models.load_model('Flowers_final.h5')

## Printing images and labels

In [ ]:
import os
from PIL import Image
import cv2
# from Keras import keras
def predict_image(model, image_path):
    
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))
    img = np.array(img).reshape(1, 128, 128, 3)
    prediction = model.predict(img)
    class_names = ["daisy", "dandelion", "rose", "sunflower", "tulip"]
    return np.argmax(prediction), class_names[np.argmax(prediction)]

def main():

    folder_path = "./flower_photos/single_prediction/"
    for file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, file)
        predicted_class, class_name = predict_image(model, image_path)
        print(f"The image {file} is classified as {class_name}")
        
        # Display the image with the prediction
        img = cv2.imread(image_path)
        cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(4, 4))
        plt.axis('off')
        plt.title(f"Predicted class: {class_name}")
        plt.imshow(img)
        plt.show()

if __name__ == "__main__":
    main()
